## Importing cleaned data

In [76]:
import pandas as pd
loans = pd.read_csv("loans_cleaned.csv")

In [79]:
loans.sample(10)

,Unnamed: 0,Sr_No,SeriousDlqin2yrs,monthly_utilization,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,...,num_30_59_dpd_new,MonthlyIncome_ind,MonthlyIncome_new,age_new,DebtRatio_new,Open_Credit_lines_new,num_90_dpd_new,Real_estate_loans_new,num_60_89_dpd_new,NumberOfDependents_new
39171,39171,39172,1,0.980940,52,3,0.589016,5498.0,15,2,...,3,1,5498.0,52,0.589016,15,2,1,0,0.0
139121,139121,139122,1,0.245455,56,0,0.725299,1841.0,10,0,...,0,1,1841.0,56,0.725299,10,0,1,0,0.0
2547,2547,2548,0,0.163463,30,0,0.235239,3200.0,12,0,...,0,1,3200.0,30,0.235239,12,0,0,0,0.0
13667,13667,13668,0,0.013810,55,0,0.359343,18800.0,11,0,...,0,1,18800.0,55,0.359343,11,0,3,0,2.0
101703,101703,101704,0,0.062188,70,0,0.019490,2000.0,5,0,...,0,1,2000.0,70,0.019490,5,0,0,0,0.0
60265,60265,60266,0,0.137551,40,0,0.249975,9900.0,10,0,...,0,1,9900.0,40,0.249975,10,0,2,0,4.0
20565,20565,20566,0,0.333133,51,0,0.460104,5200.0,12,0,...,0,1,5200.0,51,0.460104,12,0,2,0,0.0
94128,94128,94129,0,1.013497,31,0,1.550735,1428.0,6,0,...,0,1,1428.0,31,0.366508,6,0,1,0,1.0
26684,26684,26685,0,0.000000,41,0,0.538744,6000.0,7,0,...,0,1,6000.0,41,0.538744,7,0,3,0,2.0
114414,114414,114415,0,0.001496,43,4,0.492458,5833.0,8,1,...,4,1,5833.0,43,0.492458,8,1,1,3,0.0


## 3) Logistic Regression Model building

This model is easy to explain as we can clearly see the impact of each variable on the output 
-> So, due to it's interpretability, Logistic Regression is preferrred here 

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf 
import numpy as np
from sklearn.model_selection import train_test_split

X=loans.drop(["SeriousDlqin2yrs","Sr_No"],axis=1)
y=loans["SeriousDlqin2yrs"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)